In [2]:
from pyltp import Segmentor, SentenceSplitter, Postagger, Parser
from bs4 import BeautifulSoup
import json
import os
from denoter_tools3 import statistics_denoters,get_denoters_index

train_sets = os.listdir('data/train3/')
# train_sets = ['伊拉克3月恐怖袭击和暴力冲突致死592人.xml']
results_all = {}

try:
    segmentor = Segmentor()  # 初始化实例
    segmentor.load('data/ltp_data_v3.4.0/cws.model')  # 加载模型
    postagger = Postagger() # 初始化实例
    postagger.load('data/ltp_data_v3.4.0/pos.model')  # 加载模型
    parser = Parser() # 初始化实例
    parser.load('data/ltp_data_v3.4.0/parser.model')  # 加载模型

    results = {}
    statistics_postagger, statistics_parser, statistics_single_words = statistics_denoters(train_sets, segmentor, postagger, parser)
    print(sorted(statistics_postagger.items(), key = lambda item: item[1]))
    print(sorted(statistics_parser.items(), key = lambda item: item[1]))
    sum_postagger = sum([item[1] for item in statistics_postagger.items()])
    sum_parser = sum([item[1] for item in statistics_parser.items()])
    for item in statistics_postagger.items():
        statistics_postagger[item[0]] = item[1]/sum_postagger
    for item in statistics_parser.items():
        statistics_parser[item[0]] = item[1]/sum_parser 
    print(sorted(statistics_postagger.items(), key = lambda item: item[1]))
    print(sorted(statistics_parser.items(), key = lambda item: item[1]))
    
    
    for train_text in train_sets:
        print('正在训练文本：',train_text)
        with open('data/train3/' + train_text, 'r', encoding='utf-8') as f:
            xml = f.read()
        soup = BeautifulSoup(xml, 'lxml')
#         print(soup.content.get_text(strip=True))
        for denoter in soup.find_all('denoter'):
            denoter.string = '#' + denoter.string + '#'
            
        content = soup.content.get_text(strip=True).replace(' ','')#.replace('，','。')
        sents = SentenceSplitter.split(content)  # 分句
        
        for x in range(len(sents)):
            sentence = sents[x]
            denoters_range = []
            denoter_range = []
            while '#' in sentence:
                if len(denoter_range) == 0:
                    denoter_range.append(sentence.index('#'))
                elif len(denoter_range) == 1:
                    denoter_range.append(sentence.index('#')-1)
                    denoters_range.append(denoter_range)
                    denoter_range = []
                sentence = sentence.replace('#','',1)
#             print(denoters_range)
            
            words = list(segmentor.segment(sentence))
            postags = list(postagger.postag(words))  # 词性标注
            arcs = parser.parse(words, postags)  # 句法分析
            heads = [arc.head for arc in arcs]
            relations = [arc.relation for arc in arcs]
#             print(words)
#             print(postags)
#             print(heads)
#             print(relations)
            denoters_index = get_denoters_index(words, denoters_range, postags, 
                             relations, statistics_postagger, statistics_parser, statistics_single_words)
#             print(denoters_index)
            for word_index,word_value in enumerate(words):
                if relations[word_index] != 'WP':
                    all_related_nodes = []
                    parent_node = []
                    child_nodes = []
                    other_nodes = []
                    if heads[word_index] != 0:
                        parent_node.append(words[heads[word_index] - 1])
                    for head_index, head_value in enumerate(heads):
                        if head_value == (word_index + 1) and relations[head_index] != 'WP':
                            child_nodes.append(words[head_index])
                    if word_index - 1 >= 0 and relations[word_index - 1] != 'WP':
                        other_nodes.append(words[word_index - 1])
                    if word_index + 2 <= len(words) and relations[word_index + 1] != 'WP':
                        other_nodes.append(words[word_index + 1])
                    all_related_nodes = list(set(child_nodes + other_nodes + parent_node))    
                    if word_index in denoters_index:
                        if word_value not in results:
                            results[word_value] = [{}, {}, {}, {}, {}, {}, []]
                        for node in all_related_nodes:
                            if node not in results[word_value][0]:
                                results[word_value][0][node] = 1
                            else:
                                results[word_value][0][node] += 1
                        if relations[word_index] not in results[word_value][1]:
                            results[word_value][1][relations[word_index]] = 1
                        else:
                            results[word_value][1][relations[word_index]] += 1
                        if postags[word_index] not in results[word_value][2]:
                            results[word_value][2][postags[word_index]] = 1
                        else:
                            results[word_value][2][postags[word_index]] += 1    
                    
                    else:
                        if word_value not in results:
                            results[word_value] = [{}, {}, {}, {}, {}, {}, []]
                        for node in all_related_nodes:
                            if node not in results[word_value][3]:
                                results[word_value][3][node] = 1
                            else:
                                results[word_value][3][node] += 1
                        if relations[word_index] not in results[word_value][4]:
                            results[word_value][4][relations[word_index]] = 1
                        else:
                            results[word_value][4][relations[word_index]] += 1
                        if postags[word_index] not in results[word_value][5]:
                            results[word_value][5][postags[word_index]] = 1
                        else:
                            results[word_value][5][postags[word_index]] += 1
                    

        print('已训练完成文本：',train_text)
        
        
finally:
    segmentor.release()  # 释放模型
    postagger.release()  # 释放模型
    parser.release()  # 释放模型

#修剪结果集
results_reduce = []
for result in results.items():
    if len(result[1][1]) == 0:
        results_reduce.append(result[0])
for result_reduce in results_reduce:
    results.pop(result_reduce)
    
for result in results.items():
    for index in range(6):
        result[1][6].append(sum([item[1] for item in result[1][index].items()]))
    
results_all = {'results': results, 'statistics_postagger': statistics_postagger, 'statistics_parser': statistics_parser, 'statistics_single_words':statistics_single_words}
with open('data/event_extraction2_contrast2.dat','w',encoding='utf-8') as f :
    f.write(json.dumps(results_all, ensure_ascii=False))

print('训练结束！！！')


[('m', 1), ('d', 1), ('u', 1), ('z', 1), ('wp', 2), ('nt', 2), ('ns', 2), ('a', 23), ('i', 32), ('n', 551), ('v', 3303)]
[('FOB', 1), ('RAD', 1), ('WP', 2), ('CMP', 5), ('ADV', 19), ('SBV', 239), ('POB', 240), ('ATT', 421), ('HED', 792), ('VOB', 1041), ('COO', 1158)]
[('m', 0.00025516713447307985), ('d', 0.00025516713447307985), ('u', 0.00025516713447307985), ('z', 0.00025516713447307985), ('wp', 0.0005103342689461597), ('nt', 0.0005103342689461597), ('ns', 0.0005103342689461597), ('a', 0.005868844092880837), ('i', 0.008165348303138555), ('n', 0.140597091094667), ('v', 0.8428170451645828)]
[('FOB', 0.00025516713447307985), ('RAD', 0.00025516713447307985), ('WP', 0.0005103342689461597), ('CMP', 0.0012758356723653994), ('ADV', 0.0048481755549885175), ('SBV', 0.06098494513906609), ('POB', 0.06124011227353917), ('ATT', 0.10742536361316662), ('HED', 0.20209237050267925), ('VOB', 0.26562898698647613), ('COO', 0.2954835417198265)]
正在训练文本： 101国道密云段现惨祸客车农用车相撞致6人亡.xml
已训练完成文本： 101国道密云段现惨祸客车农用车相撞

已训练完成文本： 广州市一烂尾楼发生火灾.xml
正在训练文本： 广州番禺商铺火灾：一名女子被烧死.xml
已训练完成文本： 广州番禺商铺火灾：一名女子被烧死.xml
正在训练文本： 广州白云区大火：一家六口死亡.xml
已训练完成文本： 广州白云区大火：一家六口死亡.xml
正在训练文本： 广西南丹交通事故致6死10伤一人伤势较重.xml
已训练完成文本： 广西南丹交通事故致6死10伤一人伤势较重.xml
正在训练文本： 广西南梧二级路藤县特大交通事故4死9伤.xml
已训练完成文本： 广西南梧二级路藤县特大交通事故4死9伤.xml
正在训练文本： 广西境内车辆追尾致6死38伤(组图).xml
已训练完成文本： 广西境内车辆追尾致6死38伤(组图).xml
正在训练文本： 广西柳州两车相撞致10人死亡2人重伤.xml
已训练完成文本： 广西柳州两车相撞致10人死亡2人重伤.xml
正在训练文本： 广西贵港一涂料厂溶剂油罐发生火灾无人员伤亡.xml
已训练完成文本： 广西贵港一涂料厂溶剂油罐发生火灾无人员伤亡.xml
正在训练文本： 怒江兰坪发生一起交通事故.xml
已训练完成文本： 怒江兰坪发生一起交通事故.xml
正在训练文本： 意大利驻华大使强烈谴责昆明暴力恐怖袭击事件.xml
已训练完成文本： 意大利驻华大使强烈谴责昆明暴力恐怖袭击事件.xml
正在训练文本： 成都网友称震感强烈女同事当即哭泣.xml
已训练完成文本： 成都网友称震感强烈女同事当即哭泣.xml
正在训练文本： 手机充电器爆炸引发大火致2死2伤.xml
已训练完成文本： 手机充电器爆炸引发大火致2死2伤.xml
正在训练文本： 摩托车撞大货一死一伤.xml
已训练完成文本： 摩托车撞大货一死一伤.xml
正在训练文本： 斐济附近海域发生6.5级地震.xml
已训练完成文本： 斐济附近海域发生6.5级地震.xml
正在训练文本： 新加坡137人食物中毒两人死亡.xml
已训练完成文本： 新加坡137人食物中毒两人死亡.xml
正在训练文本： 新加坡发生集体食物中毒事件1人死亡.xml
已训练完成文本： 新加坡发生集体食物中毒事件1人死亡.xml
正在训练文本： 新疆伊犁5名儿童食用自制酸菜食物中毒.xml
已训练完成文本： 新疆伊犁5名儿童食用自制酸菜食物中毒.xml
正在训练文本： 新疆哈密5

In [8]:
from pyltp import Segmentor
from pyltp import SentenceSplitter
from pyltp import Postagger
from pyltp import Parser
from bs4 import BeautifulSoup
import json
import os
from denoter_tools3 import statistics_denoters,get_denoters_index
import math
def fun(x):
    return x/(1+x)
#     return (1-math.exp(-x))/(1+math.exp(-x))
#     return math.log(1+math.exp(x))
#     return (math.exp(x)-math.exp(-x))/(math.exp(x)+math.exp(-x))
test_sets = os.listdir('data/test3/')
# test_sets = ['世界杯险遇恐怖袭击警方发现犯罪组织欲炸桥.xml']
try:
    segmentor = Segmentor()  # 初始化实例
    segmentor.load('data/ltp_data_v3.4.0/cws.model')  # 加载模型
    postagger = Postagger() # 初始化实例
    postagger.load('data/ltp_data_v3.4.0/pos.model')  # 加载模型
    parser = Parser() # 初始化实例
    parser.load('data/ltp_data_v3.4.0/parser.model')  # 加载模型

    with open('data/event_extraction2_contrast2.dat','r',encoding='utf-8') as f :
        results_all = json.loads(f.read())
    results = results_all['results']
    statistics_postagger = results_all['statistics_postagger']
    statistics_parser = results_all['statistics_parser']
    statistics_single_words = results_all['statistics_single_words']
    
    total_event_count = 0
    total_event_find_count = 0
    total_event_find_right_count = 0

    for test_text in test_sets:
        print('正在测试文本：',test_text)
        with open('data/test3/' + test_text, 'r', encoding='utf-8') as f:
            xml = f.read()
        soup = BeautifulSoup(xml, 'lxml')
        event_count = len(soup.find_all('denoter'))
        event_find_count = 0
        event_find_right_count = 0
        text_with_denoter = ''
#         print(soup.content.get_text(strip=True))
        for denoter in soup.find_all('denoter'):
            denoter.string = '#' + denoter.string + '#'
        content = soup.content.get_text(strip=True).replace(' ','')#.replace('，','。')
#         print(content)
        sents = SentenceSplitter.split(content)  # 分句
        
        for x in range(len(sents)):
            
            sentence = sents[x]
            denoters_range = []
            denoter_range = []
            while '#' in sentence:
                if len(denoter_range) == 0:
                    denoter_range.append(sentence.index('#'))
                elif len(denoter_range) == 1:
                    denoter_range.append(sentence.index('#')-1)
                    denoters_range.append(denoter_range)
                    denoter_range = []
                sentence = sentence.replace('#','',1)
#             print(denoters_range)
            
            words = list(segmentor.segment(sentence))
            postags = list(postagger.postag(words))  # 词性标注
            arcs = parser.parse(words, postags)  # 句法分析
            heads = [arc.head for arc in arcs]
            relations = [arc.relation for arc in arcs]
            denoters_index = get_denoters_index(words, denoters_range, postags, 
                             relations, statistics_postagger, statistics_parser, statistics_single_words)
            candidates = set()
#             print(words)
#             print(denoters_index)
            for word_index,word_value in enumerate(words):
                if word_value in results and relations[word_index] != 'WP':
                    positive_num, negative_num = 0, 0
                    all_related_nodes = []
                    parent_node = []
                    child_nodes = []
                    other_nodes = []
                    if heads[word_index] != 0:
                        parent_node.append(words[heads[word_index] - 1])
                    for head_index, head_value in enumerate(heads):
                        if head_value == (word_index + 1) and relations[head_index] != 'WP':
                            child_nodes.append(words[head_index])
                    if word_index - 1 >= 0 and relations[word_index - 1] != 'WP':
                        other_nodes.append(words[word_index - 1])
                    if word_index + 2 <= len(words) and relations[word_index + 1] != 'WP':
                        other_nodes.append(words[word_index + 1])
                    all_related_nodes = list(set(child_nodes + other_nodes + parent_node))    
                    
                    for node in all_related_nodes:
                        if node in results[word_value][0]:
                            positive_num += fun(results[word_value][0][node])
#                             if results[word_value][0][node] > 1:
#                                 positive_num += 1.5
#                             else:
#                                 positive_num += 1
# #                             print(node,results[word_value][0][node],'=>',word_value)
                        if node in results[word_value][3]:
                            negative_num += fun(results[word_value][3][node])
#                             if results[word_value][3][node] > 1:
#                                 negative_num += 1.5
#                             else:
#                                 negative_num += 1
#                             print(node,results[word_value][3][node],'!=>',word_value)

                    if relations[word_index]  in results[word_value][1]:
                        positive_num += fun(results[word_value][1][relations[word_index]])
#                         if results[word_value][1][relations[word_index]] > 1:
#                             positive_num += 1.5
#                         else:
#                             positive_num += 1
# #                         print(relations[word_index],results[word_value][1][relations[word_index]],'<rel>',word_value)
                    if relations[word_index] in results[word_value][4]:
                        negative_num += fun(results[word_value][4][relations[word_index]])
#                         if results[word_value][4][relations[word_index]] > 1:
#                             negative_num += 1.5
#                         else:
#                             negative_num += 1
#                         print(relations[word_index],results[word_value][4][relations[word_index]],'!<rel>',word_value)
                    if postags[word_index] in results[word_value][2]:
                        positive_num += fun(results[word_value][2][postags[word_index]])
#                         if results[word_value][2][postags[word_index]] > 1:
#                             positive_num += 1.5
#                         else:
#                             positive_num += 1
                    if postags[word_index] in results[word_value][5]:
                        negative_num += fun(results[word_value][5][postags[word_index]])
#                         if results[word_value][5][postags[word_index]] > 1:
#                             negative_num += 1.5
#                         else:
#                             negative_num += 1
                        
#                     candidates.add(word_index)    
                    if positive_num - negative_num >=0:
                        candidates.add(word_index)
                        print(word_value, positive_num, negative_num,'positive_num - negative_num:', positive_num - negative_num)
                    else:
                        print('NOT' + word_value, positive_num, negative_num, 'positive_num - negative_num:', positive_num - negative_num)
            event_find_count += len(candidates)
            denoters_index_not_find = list(set(denoters_index) - set(candidates))
            for index in denoters_index_not_find:
                words[index] = '\033[1;33m' + words[index] + '\033[0m' 
            for candidate in candidates:
                if candidate in denoters_index:
                    event_find_right_count += 1
                    words[candidate] = '\033[1;32m' + words[candidate] + '\033[0m'
                else:
                    words[candidate] = '\033[1;31m' + words[candidate] + '\033[0m'
            text_with_denoter += ''.join(words)
        print(text_with_denoter)
        total_event_count += event_count
        total_event_find_count += event_find_count
        total_event_find_right_count += event_find_right_count

#         print('本文档查准率 Precision：',event_find_right_count/event_find_count)
#         print('本文档查全率 Recall：',event_find_right_count/event_count)
#         print('本文档F1值 ：',(2*pow(event_find_right_count,2))/((event_find_right_count*event_find_count)+(event_find_right_count*event_count)))

#         print('到目前为止总的查准率 Precision：',total_event_find_right_count/total_event_find_count)
#         print('到目前为止总的查全率 Recall：',total_event_find_right_count/total_event_count)
#         print('到目前为止总的F1值 ：',(2*pow(total_event_find_right_count,2))/((total_event_find_right_count*total_event_find_count)+(total_event_find_right_count*total_event_count)))

        print('已训练完成文本：',test_text,'\n\n')
finally:
    segmentor.release()  # 释放模型
    postagger.release()  # 释放模型
    parser.release()  # 释放模型
print('自动标注识别正确的触发词数：', total_event_find_right_count)
print('自动标注识别的触发词数：', total_event_find_count)
print('人工标注的触发词数：', total_event_count)
print('查准率 Precision：',total_event_find_right_count/total_event_find_count)
print('查全率 Recall：',total_event_find_right_count/total_event_count)
print('F1值 ：',(2*pow(total_event_find_right_count,2))/((total_event_find_right_count*total_event_find_count)+(total_event_find_right_count*total_event_count)))



正在测试文本： 上海一家公司70多名员工食物中毒.xml
NOT信息 0.9230769230769231 1.3333333333333335 positive_num - negative_num: -0.41025641025641035
接到 3.144505494505495 1.625 positive_num - negative_num: 1.5195054945054949
称 1.932142857142857 1.4666666666666668 positive_num - negative_num: 0.46547619047619015
NOT有 2.0 5.129974874776855 positive_num - negative_num: -3.129974874776855
NOT食物中毒 0.9722222222222222 1.4722222222222223 positive_num - negative_num: -0.5000000000000001
送 5.657390667224908 3.233333333333333 positive_num - negative_num: 2.4240573338915747
治疗 3.737653814602133 2.38974358974359 positive_num - negative_num: 1.347910224858543
NOT有 2.0 5.107171175743647 positive_num - negative_num: -3.1071711757436473
NOT危险 2.571428571428571 3.4745098039215687 positive_num - negative_num: -0.9030812324929975
前往 2.3578431372549016 0.6666666666666666 positive_num - negative_num: 1.691176470588235
食物中毒 1.928743961352657 1.9246031746031744 positive_num - negative_num: 0.0041407867494824835
看到 1.7916666666666667 0 

NOT发生 0.8 4.785102748486849 positive_num - negative_num: -3.985102748486849
事故 4.884282867999639 4.577649769585253 positive_num - negative_num: 0.3066330984143857
行驶 6.160714285714286 2.5 positive_num - negative_num: 3.6607142857142856
撞 4.017316017316017 2.5555555555555554 positive_num - negative_num: 1.4617604617604618
NOT上 1.3333333333333333 3.611813537675607 positive_num - negative_num: -2.278480204342274
死亡 3.702815934065934 2.5416666666666665 positive_num - negative_num: 1.1611492673992676
介绍 3.6033542976939206 2.388888888888889 positive_num - negative_num: 1.2144654088050317
NOT住 1.1666666666666665 1.9473684210526314 positive_num - negative_num: -0.7807017543859649
扣 1.0 0.5 positive_num - negative_num: 0.5
控制 4.501010101010101 2.659090909090909 positive_num - negative_num: 1.8419191919191915
了解 2.8819838056680163 2.761904761904762 positive_num - negative_num: 0.1200790437632544
NOT增加 0.6666666666666666 1.25 positive_num - negative_num: -0.5833333333333334
NOT发生 1.3 6.0989185379

电 3.4072971781305115 2.8055555555555554 positive_num - negative_num: 0.6017416225749561
爆炸 1.4830508474576272 0.9791666666666666 positive_num - negative_num: 0.5038841807909605
起火 2.3045112781954886 1.7282608695652173 positive_num - negative_num: 0.5762504086302713
死亡 3.9028334410687355 2.208333333333333 positive_num - negative_num: 1.6945001077354025
报道 2.8705882352941177 2.409090909090909 positive_num - negative_num: 0.4614973262032085
NOT发生 0.8 4.84979501677417 positive_num - negative_num: -4.04979501677417
火灾 2.9482539682539683 2.671957671957672 positive_num - negative_num: 0.27629629629629626
NOT大火 0.9285714285714286 3.745195195195195 positive_num - negative_num: -2.816623766623766
扑灭 4.27835429769392 0.6666666666666666 positive_num - negative_num: 3.611687631027254
烧毁 1.3333333333333335 1.0 positive_num - negative_num: 0.3333333333333335
发现 4.5284391534391535 2.966666666666667 positive_num - negative_num: 1.5617724867724867
NOT火灾 2.4305555555555554 2.5893246187363834 positive_num

说 1.9652738565782044 1.625 positive_num - negative_num: 0.3402738565782044
淹没 0.5 0 positive_num - negative_num: 0.5
中新网5月27日电据美联社报道，27日早晨发生在印尼中爪哇省日惹市附近的里氏6.2级强震已造成至少309人死亡，数百人受伤，大批房屋被夷为平地。距离震中最近的穆罕默迪亚医院官员库斯马万托在接受采访时说：“请告诉中央政府赶紧提供援助，我们这里需要帮助。有大量人员伤亡。房屋被夷为平地。许多人需要疏散。”贝斯西达医院管理太平间的工作人员苏班迪则说：“我们已经被尸体淹没。”(章田)
已训练完成文本： 印尼地震将大批房屋夷为平地太平间被尸体淹没.xml 


正在测试文本： 印尼爪哇省发生6.2级地震已至少造成51人死亡.xml
消息 1.6973684210526314 0.9444444444444444 positive_num - negative_num: 0.752923976608187
表示 2.630952380952381 2.0999999999999996 positive_num - negative_num: 0.5309523809523813
NOT发生 1.3 5.481081774297279 positive_num - negative_num: -4.1810817742972795
地震 3.937658814603652 3.607900432900433 positive_num - negative_num: 0.3297583817032188
倒塌 1.7745098039215685 1.6 positive_num - negative_num: 0.17450980392156845
死亡 3.9616037846492054 2.208333333333333 positive_num - negative_num: 1.7532704513158723
报道 2.8705882352941177 2.409090909090909 positive_num - negative_num: 0.4614973262032085
称 4.714285714285714 1.8 posit

吃 5.844444444444444 1.5416666666666665 positive_num - negative_num: 4.302777777777777
出现 4.723484848484848 2.3666666666666667 positive_num - negative_num: 2.356818181818181
NOT腹痛 0.5 1.8571428571428572 positive_num - negative_num: -1.3571428571428572
NOT急诊 1.0 2.0 positive_num - negative_num: -1.0
治疗 1.941132075471698 1.7230769230769232 positive_num - negative_num: 0.2180551523947749
出院 3.38974358974359 0 positive_num - negative_num: 3.38974358974359
表示 1.9116541353383458 1.7666666666666666 positive_num - negative_num: 0.14498746867167922
吃 4.711111111111111 2.875 positive_num - negative_num: 1.8361111111111112
NOT反映 0.6666666666666666 1.5 positive_num - negative_num: -0.8333333333333334
NOT感觉 1.5238095238095237 1.7424242424242424 positive_num - negative_num: -0.2186147186147187
出现 3.606818181818182 1.8666666666666667 positive_num - negative_num: 1.7401515151515152
NOT恶心 1.0 2.209090909090909 positive_num - negative_num: -1.209090909090909
NOT腹痛 1.0 2.107142857142857 positive_num - neg

...

称 1.9404761904761905 1.3 positive_num - negative_num: 0.6404761904761904
进食 2.3333333333333335 0.5 positive_num - negative_num: 1.8333333333333335
食物中毒 1.8952991452991452 1.8952991452991452 positive_num - negative_num: 0.0
NOT事发 1.861904761904762 1.9357142857142857 positive_num - negative_num: -0.07380952380952377
NOT介入 2.0 2.9642857142857144 positive_num - negative_num: -0.9642857142857144
调查 2.8369946091644205 1.4375 positive_num - negative_num: 1.3994946091644205
坐 2.0 1.3333333333333333 positive_num - negative_num: 0.6666666666666667
NOT上 0 2.916837606837607 positive_num - negative_num: -2.916837606837607
NOT治疗 2.231132075471698 2.523076923076923 positive_num - negative_num: -0.2919448476052251
NOT上 0 1.8937062937062938 positive_num - negative_num: -1.8937062937062938
NOT有 1.25 5.644179894179894 positive_num - negative_num: -4.394179894179894
安放 1.1666666666666665 0 positive_num - negative_num: 1.1666666666666665
本网讯（记者孟祥超）17日下午，石家庄市卓达服装城一工厂20余名工人陆续出现四肢无力、恶心、呕吐等食物中毒症状，当晚被送医院救治。多名患者

NOT袭击 1.9007936507936507 2.7271099744245526 positive_num - negative_num: -0.8263163236309019
9月18日，位于索马里过渡政府所在地索南部城市拜多阿的议会大楼前发生汽车炸弹爆炸事件，造成至少6人死亡和多人受伤，袭击目标索临时总统优素福则幸免于难。当优素福的车队行驶至议会大厦门口时，一辆装有炸药的汽车在总统车队附近爆炸，至少有6人死亡，其中包括3名议会保安和3名平民。爆炸还造成多人受伤，但目前还不能确定伤者人数，优素福本人并未在袭击中受伤。目击者称，爆炸引发的火球冲入天空，周围多辆汽车被毁。在这起爆炸发生后大约15分钟，议会大楼附近又发生了第二起爆炸，伤亡情况尚不清楚。索马里过渡政府发言人迪纳里当天说，可以确信袭击的目标就是优素福总统。他说，爆炸发生前，优素福刚在过渡议会发表完讲话。索马里过渡政府外长伊斯梅尔・胡雷・布巴说，汽车炸弹由遥控器引爆，明显“具有‘基地’组织袭击的特征”。
已训练完成文本： 索马里恐怖袭击事件.xml 


正在测试文本： 约旦首都安曼连环爆炸.xml
NOT发生 1.4666666666666668 6.035387023990421 positive_num - negative_num: -4.5687203573237545
爆炸 4.340893984712529 3.649508060692271 positive_num - negative_num: 0.6913859240202576
NOT袭击 2.9097222222222223 3.5250995134896064 positive_num - negative_num: -0.615377291267384
NOT事件 1.6904761904761905 5.396535996714207 positive_num - negative_num: -3.7060598062380166
死亡 4.937213540746766 2.208333333333333 positive_num - negative_num: 2.7288802074134333
受伤 3.9381661470750045 1.4411764705882353 positive_num - nega

扑救 2.114035087719298 1.4 positive_num - negative_num: 0.714035087719298
派出 1.7424242424242424 1.0 positive_num - negative_num: 0.7424242424242424
NOT到场 0.6666666666666666 2.125 positive_num - negative_num: -1.4583333333333335
增援 1.75 0.6666666666666666 positive_num - negative_num: 1.0833333333333335
奋战 1.0 1.0 positive_num - negative_num: 0.0
NOT火灾 1.8611111111111112 3.0886243386243386 positive_num - negative_num: -1.2275132275132274
扑救 2.3045112781954886 1.9 positive_num - negative_num: 0.4045112781954887
本报讯3月27日晚上10点至3月28日凌晨2点，鄂尔多斯市达拉特旗树林召镇210国道35公里周边村庄一村民家蔬菜大棚突发大火，14家村民家草垛起火，情况紧急，请求扑救。消防指挥员在前往火灾现场的途中，电话指导报警人组织村民先对火势进行有效控制，抵达火灾现场后，只见现场周边浓烟滚滚，一片火海。了解情况后，指挥员立即指挥消防官兵进行扑救。10点半左右，达旗新能能源企业消防队派出2辆消防车到场增援。经过全体人员连续4小时的奋战，3月28日凌晨2点，15起火灾被成功扑救。
已训练完成文本： 草垛夜间连连失火消防4小时扑救15起.xml 


正在测试文本： 莫斯科一市场爆炸至少10人死亡.xml
电 4.754320987654321 2.0555555555555554 positive_num - negative_num: 2.6987654320987655
宣布 2.3321678321678325 1.1666666666666665 positive_num - negative_num: 1.165501165501166
NOT发生 1.3 4.812

说 4.498607189911538 2.125 positive_num - negative_num: 2.373607189911538
抓获 1.1666666666666665 0 positive_num - negative_num: 1.1666666666666665
NOT调查 0.9905660377358491 1.6041666666666665 positive_num - negative_num: -0.6136006289308175
证实 1.7857142857142856 0.5 positive_num - negative_num: 1.2857142857142856
NOT事件 0.8571428571428571 3.3980681665997055 positive_num - negative_num: -2.5409253094568482
NOT有 0.75 3.215958605664488 positive_num - negative_num: -2.465958605664488
证实 1.7857142857142856 0.5 positive_num - negative_num: 1.2857142857142856
NOT组织 2.216666666666667 2.7333333333333334 positive_num - negative_num: -0.5166666666666666
NOT实施 1.8 2.2083333333333335 positive_num - negative_num: -0.40833333333333344
爆炸案 1.3333333333333333 0.5 positive_num - negative_num: 0.8333333333333333
NOT发生 0.8 3.624839485790083 positive_num - negative_num: -2.8248394857900827
爆炸 2.895936509889569 2.3125 positive_num - negative_num: 0.5834365098895691
炸 1.3333333333333333 0 positive_num - negative

烧毁 2.5 0.5 positive_num - negative_num: 2.0
NOT损失 0.5 3.0375 positive_num - negative_num: -2.5375
2014年1月11日凌晨1时27分，位于云南迪庆藏族自治州州府香格里拉县的独克宗古城发生大火，起火原因已经查明，为客栈管理者用火不慎引起火灾。截至2014年1月11日17时46分，古城内明火已经被扑灭，消防官兵还在搜寻是否有人员被困。据初步统计火灾烧毁房屋100多栋，初步测算已造成经济损失达1亿多元人民币。
已训练完成文本： 香格里拉大火事件.xml 


自动标注识别正确的触发词数： 1140
自动标注识别的触发词数： 1390
人工标注的触发词数： 1490
查准率 Precision： 0.8201438848920863
查全率 Recall： 0.7651006711409396
F1值 ： 0.7916666666666666
